In [1]:
pip install opencv-python numpy tensorflow tensorflow-hub ultralytics

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade tensorflow tensorflow-hub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2  # OpenCV ile görüntü işleme
import numpy as np  # Sayısal işlemler için
import tensorflow as tf  # TensorFlow kullanımı için
import tensorflow_hub as hub  # Model yükleme
from ultralytics import YOLO  # YOLO modeli


MODEL_PATH = r"C:\Users\abdlk\Downloads\PROJE\deeplabv3"  # Raw string formatı

print("DeepLabV3 modelini yükleniyor")

try:
    model = hub.load(MODEL_PATH) 
    print("Model başarıyla yüklendi")
except Exception as e:
    print("HATA: Model yüklenemedi. Lütfen MODEL_PATH yolunu kontrol et!")
    print(e)
    exit()

# YOLO Nesne Algılama Modeli
print("YOLO modelini yüklüyoruz")
yolo_model = YOLO("yolov8n.pt")  # YOLOv8 

# Video Kaynağını Aç 
video_kaynagi = cv2.VideoCapture("people.mp4")

if not video_kaynagi.isOpened():
    print("HATA: Video kaynağına erişilemedi!")
    exit()

while True:
    ret, kare = video_kaynagi.read()  # Videodan bir kare oku
    if not ret:
        break 

    # YOLO Kullanarak Nesne Algılama
    sonuclar = yolo_model(kare)

    for sonuc in sonuclar: 
        for kutu in sonuc.boxes:
            x1, y1, x2, y2 = map(int, kutu.xyxy[0])
            etiket = sonuc.names[int(kutu.cls[0])]
            
            nesne = kare[y1:y2, x1:x2]
            nesne = cv2.resize(nesne, (513, 513))
            nesne = np.expand_dims(nesne, axis=0) / 255.0  # Normalizasyon yap
            
            # Maskeleme Yap
            tahmin = model.signatures["default"](tf.convert_to_tensor(nesne))
            maske = tahmin["semantic_predictions"].numpy()[0]

            # Maskeyi Uygun Boyuta Getir
            maske = cv2.resize(maske.astype(np.uint8), (x2 - x1, y2 - y1))

            # Arka Planı Siyah Yaparak Maske Uygula
            kare[y1:y2, x1:x2][maske == 0] = (0, 0, 0)  # Siyah arka plan uygula

            # Etiket ve Çerçeve Ekle
            cv2.rectangle(kare, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Nesnenin çevresine çerçeve çiz
            cv2.putText(kare, etiket, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imshow("Maskeleme ve Nesne Etiketleme", kare)

    # Çıkış İçin 'q' Tuşuna Basılmasını Bekle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Pencereleri Kapat
video_kaynagi.release()
cv2.destroyAllWindows()



DeepLabV3 modelini yükleniyor...

HATA: Model yüklenemedi. Lütfen MODEL_PATH yolunu kontrol et!
Trying to load a model of incompatible/unknown type. 'C:\Users\abdlk\Downloads\PROJE\deeplabv3' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.
YOLO modelini yüklüyoruz...

0: 384x640 4 persons, 1 chair, 1 potted plant, 2 laptops, 86.2ms
Speed: 4.9ms preprocess, 86.2ms inference, 7.7ms postprocess per image at shape (1, 3, 384, 640)


NameError: name 'model' is not defined

In [1]:
import os

MODEL_PATH = "C:/Users/abdlk/Downloads/PROJE/deeplabv3"

if not os.path.exists(MODEL_PATH):
    print("HATA: MODEL_PATH hatalı, klasör bulunamadı!")
elif not os.path.exists(os.path.join(MODEL_PATH, "saved_model.pb")):
    print("HATA: 'saved_model.pb' dosyası klasörde bulunamadı!")
else:
    print("Model klasörü ve dosyalar doğru şekilde mevcut!")


HATA: 'saved_model.pb' dosyası klasörde bulunamadı!
